<a href="https://colab.research.google.com/github/nerdk312/Model-based-RL/blob/master/Model_based_STDIM_V2_090320.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

In [1]:
!pip install git+git://github.com/mila-iqia/atari-representation-learning.git
!pip install git+git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/mila-iqia/atari-representation-learning.git to /tmp/pip-req-build-89ga4jat
  Running command git clone -q git://github.com/mila-iqia/atari-representation-learning.git /tmp/pip-req-build-89ga4jat
  Created wheel for atariari: filename=atariari-0.0.1-cp36-none-any.whl size=46584 sha256=261abdbc46ecd8bbb5efe895345397b7d30ff11a09c322063fe7094556615827
  Stored in directory: /tmp/pip-ephem-wheel-cache-ckhaz94b/wheels/3d/69/51/5e436e5ae566c5b4dec5c53e65396d516459877a42a11d7aa4
Successfully built atariari
  Cloning git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail to /tmp/pip-req-build-5dce9ssj
  Running command git clone -q git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail /tmp/pip-req-build-5dce9ssj
  Created wheel for a2c-ppo-acktr: filename=a2c_ppo_acktr-0.0.1-cp36-none-any.whl size=18833 sha256=f976dc8fee9caa79f55b6de2623618ed53226c68e19f409854a55e220c09ba82
  Stored in directory: /tmp/pip-ephem-wheel-cache-k8y03gzw/wheels/91/52/02/ec5c530fd76d56a66

In [2]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

from __future__ import print_function
import pickle
import sys
sys.path.append('/content/gdrive/My Drive/Unsupervised_state_representation/atariari')

import wandb

import argparse

import random
import torch
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import RandomSampler, BatchSampler
from torchvision import transforms
import torchvision.transforms.functional as TF

from tqdm import tqdm
import datetime
import time

from atariari.methods.utils import calculate_accuracy, Cutout, EarlyStopping
from atariari.methods.trainer import Trainer
from atariari.benchmark.episodes import get_episodes
from atariari.benchmark.envs import *
from atariari.methods.utils import get_argparser
#from benchmark import *
#from methods import utils

# Imported required for the Model-based RL
from sklearn.preprocessing import StandardScaler

In [5]:
!wandb login ####################

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [6]:
parser = get_argparser()
args = parser.parse_args("")
all_defaults = {}
for key in vars(args):
    all_defaults[key] = parser.get_default(key)

all_defaults['env_name'] = 'MsPacmanNoFrameskip-v4'

wandb.init(entity="nerdk312", project="STDIM",config=all_defaults)

W&B Run: https://app.wandb.ai/nerdk312/STDIM/runs/2jxg9nvq

In [7]:
all_defaults

{'batch_size': 64,
 'beta': 1.0,
 'checkpoint_index': -1,
 'collect_mode': 'random_agent',
 'color': False,
 'cuda_id': 0,
 'encoder_type': 'Nature',
 'end_with_relu': False,
 'entropy_threshold': 0.6,
 'env_name': 'MsPacmanNoFrameskip-v4',
 'epochs': 100,
 'feature_size': 256,
 'gru_layers': 2,
 'gru_size': 256,
 'linear': True,
 'lr': 0.0003,
 'method': 'infonce-stdim',
 'naff_fc_size': 2048,
 'no_downsample': True,
 'num_frame_stack': 1,
 'num_processes': 8,
 'num_rew_evals': 10,
 'num_runs': 1,
 'patience': 15,
 'pred_offset': 1,
 'pretraining_steps': 100000,
 'probe_collect_mode': 'random_agent',
 'probe_lr': 0.0003,
 'probe_steps': 50000,
 'seed': 42,
 'sequence_length': 100,
 'steps_end': 99,
 'steps_start': 0,
 'steps_step': 4,
 'train_encoder': True,
 'use_multiple_predictors': False,
 'wandb_entity': None,
 'wandb_proj': 'atari-reps',
 'weights_path': 'None'}

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from a2c_ppo_acktr.utils import init
import time
from atariari.benchmark.utils import download_run
from atariari.benchmark.episodes import checkpointed_steps_full_sorted
import os

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class Conv2dSame(torch.nn.Module): # Nawid - Performs convolution in the same way as 'same' tensorflow format I assume
    def __init__(self, in_channels, out_channels, kernel_size, bias=True, padding_layer=nn.ReflectionPad2d):
        super().__init__()
        ka = kernel_size // 2
        kb = ka - 1 if kernel_size % 2 == 0 else ka
        self.net = torch.nn.Sequential(
            padding_layer((ka, kb, ka, kb)),
            torch.nn.Conv2d(in_channels, out_channels, kernel_size, bias=bias)
        )

    def forward(self, x):
        return self.net(x)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            Conv2dSame(in_channels, out_channels, 3),
            nn.ReLU(),
            Conv2dSame(in_channels, out_channels, 3)
        )

    def forward(self, x):
        residual = x
        out = self.block(x)
        out += residual
        out = F.relu(out)
        return out


class ImpalaCNN(nn.Module): # Nawid -  CNN architecture in the Impala paper
    def __init__(self, input_channels, args):
        super(ImpalaCNN, self).__init__()
        self.hidden_size = args['feature_size']
        self.depths = [16, 32, 32, 32]
        self.downsample = not args['no_downsample']
        self.layer1 = self._make_layer(input_channels, self.depths[0])
        self.layer2 = self._make_layer(self.depths[0], self.depths[1])
        self.layer3 = self._make_layer(self.depths[1], self.depths[2])
        self.layer4 = self._make_layer(self.depths[2], self.depths[3])
        if self.downsample:
            self.final_conv_size = 32 * 9 * 9
        else:
            self.final_conv_size = 32 * 12 * 9
        self.final_linear = nn.Linear(self.final_conv_size, self.hidden_size)
        self.flatten = Flatten()
        self.train()

    def _make_layer(self, in_channels, depth): # Nawid-  Used to make a layer
        return nn.Sequential(
            Conv2dSame(in_channels, depth, 3),
            nn.MaxPool2d(3, stride=2),
            nn.ReLU(),
            ResidualBlock(depth, depth),
            nn.ReLU(),
            ResidualBlock(depth, depth)
        )

    @property
    def local_layer_depth(self):
        return self.depths[-2]

    def forward(self, inputs, fmaps=False):
        #print(inputs.size())
        f5 = self.layer3(self.layer2(self.layer1(inputs))) # Nawid -Uses the output of the third layer and then sees whether we want to downsample or not
        
        if not self.downsample:
            out = self.layer4(f5)
            
        else:
            out = f5
            
        
        #print('before out size',out.size())
        out = F.relu(self.final_linear(self.flatten(out))) # Nawid- global feature vector
        #print('after out size', out.size())

        if fmaps:
            return {
                'f5': f5.permute(0, 2, 3, 1), # Nawid - Make the channels in the last dimension
                'out': out
            }

        return out

class NatureCNN(nn.Module): # Nawid - Nature CNN

    def __init__(self, input_channels, args):
        super().__init__()
        self.feature_size = args['feature_size']
        self.hidden_size = self.feature_size
        self.downsample = not args['no_downsample']
        self.input_channels = input_channels
        self.end_with_relu = args['end_with_relu']
        self.args = args
        init_ = lambda m: init(m,
                               nn.init.orthogonal_,
                               lambda x: nn.init.constant_(x, 0),
                               nn.init.calculate_gain('relu'))
        self.flatten = Flatten()

        if self.downsample:
            self.final_conv_size = 32 * 7 * 7
            self.final_conv_shape = (32, 7, 7)
            self.main = nn.Sequential(
                init_(nn.Conv2d(input_channels, 32, 8, stride=4)),
                nn.ReLU(),
                init_(nn.Conv2d(32, 64, 4, stride=2)),
                nn.ReLU(),
                init_(nn.Conv2d(64, 32, 3, stride=1)),
                nn.ReLU(),
                Flatten(),
                init_(nn.Linear(self.final_conv_size, self.feature_size)),
                #nn.ReLU()
            )
        else:
            self.final_conv_size = 64 * 9 * 6
            self.final_conv_shape = (64, 9, 6)
            self.main = nn.Sequential(
                init_(nn.Conv2d(input_channels, 32, 8, stride=4)),
                nn.ReLU(),
                init_(nn.Conv2d(32, 64, 4, stride=2)),
                nn.ReLU(),
                init_(nn.Conv2d(64, 128, 4, stride=2)),
                nn.ReLU(),
                init_(nn.Conv2d(128, 64, 3, stride=1)),
                nn.ReLU(),
                Flatten(),
                init_(nn.Linear(self.final_conv_size, self.feature_size)),
                #nn.ReLU()
            )
        self.train()

    @property
    def local_layer_depth(self):
        return self.main[4].out_channels

    def forward(self, inputs, fmaps=False):
        f5 = self.main[:6](inputs)
        f7 = self.main[6:8](f5)
        out = self.main[8:](f7)
        if self.end_with_relu:
            assert self.args.method != "vae", "can't end with relu and use vae!"
            out = F.relu(out)
        if fmaps: # Nawid - obtains the different feature maps as well as global feature vector
            return {
                'f5': f5.permute(0, 2, 3, 1),
                'f7': f7.permute(0, 2, 3, 1),
                'out': out
            }
        return out



class PPOEncoder(nn.Module):
    def __init__(self, env_name, checkpoint_index):
        super().__init__()
        checkpoint_step = checkpointed_steps_full_sorted[checkpoint_index]
        filepath = download_run(env_name, checkpoint_step)
        while not os.path.exists(filepath):
            time.sleep(5)

        self.masks = torch.zeros(1, 1)
        self.ppo_model, ob_rms = torch.load(filepath, map_location=lambda storage, loc: storage)

    def forward(self, x):
        _, _, _, _, feature_vectors, _ = self.ppo_model.act(x,
                                                            None,
                                                            self.masks,
                                                            deterministic=False)
        return feature_vectors


# Model-based RL

In [0]:
class NNDynamicModel(nn.Module):
    '''
    Model that predicts the next state, given the current state and action
    '''
    def __init__(self, input_dim, obs_output_dim):
        super(NNDynamicModel,self).__init__()
        self.Linear1 = nn.Linear(input_dim,512)
        self.BN1 = nn.BatchNorm1d(num_features = 512)
        self.Linear2 = nn.Linear(512,256)
        self.BN2 = nn.BatchNorm1d(num_features=256)
        self.Linear3 = nn.Linear(256, obs_output_dim) 
        

    def forward(self,x):
        x = self.Linear1(x.float())
        x = self.BN1(x)
        x = F.relu(x)
        x = self.Linear2(x)
        x = self.BN2(x)
        x = F.relu(x)
        output = self.Linear3(x)        
        return output

In [0]:
class NNRewardModel(nn.Module):
    '''
    Model that predict the reward given the current state and action
    '''
    def __init__(self, input_dim, reward_output_dim):
        super(NNRewardModel, self).__init__()
        
        self.Linear1 = nn.Linear(input_dim,512)
        self.BN1 = nn.BatchNorm1d(num_features = 512)
        self.Linear2 = nn.Linear(512,256)
        self.BN2 = nn.BatchNorm1d(num_features=256)
        self.Linear3 = nn.Linear(256, reward_output_dim) 
        
    def forward(self,x):
        x = self.Linear1(x.float())
        #print('first linear x size',x.size())
        x = self.BN1(x)
        x = F.relu(x)
        x = self.Linear2(x)
        #print('second linear x size',x.size())
        x = self.BN2(x)
        x = F.relu(x)
        output = self.Linear3(x)
        #print('third linear x size',output.size())        
        return output

In [0]:
def gather_random_trajectories(num_traj, env_name,encoder):
    '''
    Run num_traj random trajectories to gather information about the next state and reward.
    Data used to train the models in a supervised way.
    '''

    dataset_random = []
    game_rewards = []
    with torch.no_grad():
        for n in range(num_traj):
            print(n)        
            env = make_vec_envs(ENV_NAME,1,workers,downsample=False) # Nawid- Makes several different vectorised environment
            obs = env.reset()
            #print('does observation work',obs.shape)
        
            state = encoder(obs.float().to(device) / 255)
        
            while True:
                sampled_action = torch.tensor([env.action_space.sample() for i in range(workers)]).unsqueeze(dim=1) # Nawid - This needs to be used for the case where the environment is vectorised
                new_obs, reward, done, _ = env.step(sampled_action)
            
                new_state = encoder(new_obs.float().to(device) / 255)
                #print('new_state',new_state.size())
                #print('reward', reward.size())
                #print('sampled_action', sampled_action.size())
                dataset_random.append([state.cpu(), new_state.cpu(), reward, done,sampled_action.float()]) # Nawid - Appends the state instead of the observation, changed action to float in order to concatenate with the observation tensor to use as training data, and need to move the state to cpu in order to change it to numpy
                #print('dataset_random',dataset_random.shape) 
                #dataset_random.append([obs, new_obs, reward, done,sampled_action]) # Nawid - Appends the state instead of the observation

                obs = new_obs
                game_rewards.append(reward)

                if done:
                    env.close()
                    break

        # print some stats
        print('Mean R:',np.round(np.sum(game_rewards)/num_traj,2), 'Max R:', np.round(np.max(game_rewards),2), np.round(len(game_rewards)/num_traj))

    return dataset_random

In [0]:
def model_MSEloss(y_truth, y_pred, device):
    '''
    Compute the MSE (Mean Squared Error)
    '''
    y_truth = torch.FloatTensor(np.array(y_truth)).to(device)
    return F.mse_loss(y_pred.view(-1).float(), y_truth.view(-1))

In [0]:
def train_dyna_model(random_dataset, rl_dataset, env_model, rew_model, batch_size, max_model_iter, num_examples_added, ENV_LEARNING_RATE, REW_LEARNING_RATE, device):
    '''
    Train the two models that predict the next state and the expected reward
    '''

    env_optimizer = optim.Adam(env_model.parameters(), lr=ENV_LEARNING_RATE)
    rew_optimizer = optim.Adam(rew_model.parameters(), lr=REW_LEARNING_RATE)

    if len(rl_dataset) > 0:
        '''
        # To use only a fraction of the random dataset
        rand = np.arange(len(random_dataset))
        np.random.shuffle(rand)
        rand = rand[:int(len(rl_dataset)*0.8)] # 80% of rl dataset
        d_concat = np.concatenate([np.array(random_dataset)[rand], rl_dataset], axis=0)'''

        # Concatenate the random dataset with the RL dataset. Used only in the aggregation iterations
        d_concat = np.concatenate([random_dataset, rl_dataset], axis=0)
    else:
        d_concat = np.array(random_dataset)

    # Split the dataset into train(80%) and test(20%)
    D_train = d_concat[:int(-num_examples_added*1/5)]
    D_valid = d_concat[int(-num_examples_added*1/5):]

    print("len(D):", len(d_concat), 'len(Dtrain)', len(D_train))

    # Shuffle the dataset
    sff = np.arange(len(D_train))
    np.random.shuffle(sff)
    D_train = D_train[sff]

    print('does it work at this point')
    # Create the input and output for the train
    #for obs,_,_,_,act in D_train:
    #    concatenated_array =  np.concatenate([obs,act],axis = 1)
    
    #obs_train = D_train[:,0] 
    #n_obs_train = D_train[:,1]
    #reward_train = D_train[:,2]
    #act_train = D_train[:,3]
 
  
    X_train = torch.empty(size=(len(D_train),feature_size + 1)) # env.action_space.n))
    y_env_train = torch.empty(size=(len(D_train),feature_size))
    y_rew_train = torch.empty(size=(len(D_train),1))
    

    for i, (obs,n_obs,rew,_,act) in enumerate(D_train):
        X_train[i] = torch.cat((obs,act),axis=1)
        y_env_train[i] = n_obs - obs # Nawid - Change of state is the label for the model training # y(state) = s(t+1) - s(t)
        y_rew_train[i] = rew

    print('X_train', X_train.size())
    print('y_env_train', y_env_train.size())
    print('y_rew_train',y_rew_train.size())
    
    X_valid = torch.empty(size=(len(D_train),feature_size + 1)) # env.action_space.n))
    y_env_valid = torch.empty(size=(len(D_train),feature_size))
    y_rew_valid = torch.empty(size=(len(D_train),1))

    for i, (obs,n_obs,rew,_,act) in enumerate(D_valid):
        X_valid[i] = torch.cat((obs,act),axis=1)
        y_env_valid[i] = n_obs - obs # Nawid - Change of state is the label for the model training # y(state) = s(t+1) - s(t)
        y_rew_valid[i] = rew


    # Standardize the input features by removing the mean and scaling to unit variance
    input_scaler = StandardScaler()
    X_train = input_scaler.fit_transform(X_train)
    X_valid = input_scaler.transform(X_valid)

    # Standardize the outputs by removing the mean and scaling to unit variance

    env_output_scaler = StandardScaler()
    y_env_train = env_output_scaler.fit_transform(y_env_train)
    y_env_valid = env_output_scaler.transform(y_env_valid)

    rew_output_scaler = StandardScaler()
    y_rew_train = rew_output_scaler.fit_transform(y_rew_train)
    y_rew_valid = rew_output_scaler.transform(y_rew_valid)
    print('does it work after setting up the scaling')
    # store all the scalers in a variable to later uses
    norm = (input_scaler, env_output_scaler, rew_output_scaler)

    losses_env = []
    losses_rew = []

    # go through max_model_iter supervised iterations
    for it in tqdm(range(max_model_iter)):
        # create mini batches of size batch_size
        for mb in range(0, len(X_train), batch_size):
            print('Does it work before making batches')
            if len(X_train) > mb+BATCH_SIZE:
                X_mb = X_train[mb:mb+BATCH_SIZE]
                print('Does it work after making batches')
                
                y_env_mb = y_env_train[mb:mb+BATCH_SIZE]
                y_rew_mb = y_rew_train[mb:mb+BATCH_SIZE]


                print('X_mb size',X_mb.shape)
                print('y_env_mb size',y_env_train.shape)
                print('y_rew_mb size',y_rew_mb.shape)

                # Add gaussian noise with mean 0 and variance 0.0001 as in the paper
                X_mb += np.random.normal(loc=0, scale=0.001, size=X_mb.shape)

                ## Optimization of the 'env_model' neural net

                env_optimizer.zero_grad()
                # forward pass of the model to compute the output
                print('Does it work before prediction')
                pred_state = env_model(torch.tensor(X_mb).to(device))
                print('Does it work after prediction')
                # compute the MSE loss
                loss = model_MSEloss(y_env_mb, pred_state, device)
                print('Does it work after loss')

                if it == (max_model_iter - 1):
                    losses_env.append(loss.cpu().detach().numpy())

                # backward pass
                loss.backward()
                # optimization step
                env_optimizer.step()


                ## Optimization of the 'rew_model' neural net
                rew_optimizer.zero_grad()
                # forward pass of the model to compute the output
                print('Does it work before reward prediction')
                pred_rew = rew_model(torch.tensor(X_mb).to(device))
                print('Does it work after reward prediction')
                # compute the MSE loss
                loss = model_MSEloss(y_rew_mb, pred_rew, device)

                if it == (max_model_iter - 1):
                    losses_rew.append(loss.cpu().detach().numpy())
                # backward pass
                loss.backward()
                # optimization step
                rew_optimizer.step()

        # Evalute the models every 10 iterations and print the losses
        if it % 10 == 0:
          env_model.eval()
          rew_model.eval()

          pred_state = env_model(torch.tensor(X_valid).to(device))
          pred_rew = rew_model(torch.tensor(X_valid).to(device))
          env_model.train(True)
          rew_model.train(True)

          valid_env_loss = model_MSEloss(y_env_valid, pred_state, device)
          valid_rew_loss = model_MSEloss(y_rew_valid, pred_rew, device)

          print('..', it, valid_env_loss.cpu().detach().numpy(), valid_rew_loss.cpu().detach().numpy())


    ## Evaluate the MSE losses

    env_model.eval()
    rew_model.eval()

    pred_state = env_model(torch.tensor(X_valid).to(device))
    pred_rew = rew_model(torch.tensor(X_valid).to(device))
    env_model.train(True)
    rew_model.train(True)

    valid_env_loss = model_MSEloss(y_env_valid, pred_state, device)
    valid_rew_loss = model_MSEloss(y_rew_valid, pred_rew, device)

    return np.mean(losses_env), np.mean(losses_rew), valid_env_loss.cpu().detach().numpy(), valid_rew_loss.cpu().detach().numpy(), norm

In [0]:
def multi_model_based_control(env_model, rew_model, real_obs, num_sequences, horizon_length, sample_action, norm, device):
    '''
    Use a random-sampling shooting method, generating random action sequences. The first action with the highest reward of the entire sequence is returned
    '''
    best_reward = -1e9
    best_next_action = []

    input_scaler, env_output_scaler, rew_output_scaler = norm

    m_obs = np.array([real_obs for _ in range(num_sequences)])

    # array that contains the rewards for all the sequence
    unroll_rewards = np.zeros((num_sequences, 1))
    first_sampled_actions = []

    env_model.eval()
    rew_model.eval()

    ## Create a batch of size 'num_sequences' (number of trajectories) to roll the models 'horizon_length' times.
    ## i.e. roll a given number of trajectories in a single batch (to increase speed)

    for t in range(horizon_length):
      # sampled actions for each sequence
      sampled_actions = [sample_action() for _ in range(num_sequences)]
      # scale the input
      sampled_actions = np.expand_dims(sampled_actions,axis=1)   
      models_input = input_scaler.transform(np.concatenate([m_obs, sampled_actions], axis=1))
      # compute the next state for each sequence
      pred_obs = env_model(torch.tensor(models_input).to(device))
      # and the reward
      pred_rew = rew_model(torch.tensor(models_input).to(device))

      # inverse scaler transofrmation
      pred_obs = env_output_scaler.inverse_transform(pred_obs.cpu().detach().numpy())
      # and add previous observation
      m_obs = pred_obs + m_obs

      assert(pred_rew.cpu().detach().numpy().shape == unroll_rewards.shape)

      # sum of the expected rewards
      unroll_rewards += pred_rew.cpu().detach().numpy()

      if t == 0:
        first_sampled_actions = sampled_actions

    env_model.train(True)
    rew_model.train(True)

    # Best the position of the sequence with the higher reward
    arg_best_reward = np.argmax(unroll_rewards)
    best_sum_reward = unroll_rewards[arg_best_reward].squeeze()
    # take the first action of this sequence
    best_action = first_sampled_actions[arg_best_reward].squeeze()

    return best_action, best_sum_reward


In [0]:
ENV_NAME = 'MsPacmanNoFrameskip-v4'

feature_size = all_defaults['feature_size'] # Nawid- Dimensionality of the representation
workers = 1 # Nawid - Choosing the number of workers for the network

# Main loop hyperp
AGGR_ITER = 3
STEPS_PER_AGGR = 20000

# Random MB Hyperp
NUM_RAND_TRAJECTORIES = 2 #1000

# cuda or cpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu' )

# Supervised Model Hyperp
ENV_LEARNING_RATE = 1e-3
REW_LEARNING_RATE = 1e-3
BATCH_SIZE = 512
TRAIN_ITER_MODEL = 55

# Controller Hyper
HORIZON_LENGTH = 10
NUM_ACTIONS_SEQUENCES = 20000

observation_channels = 1

In [28]:
def main():

    torch.cuda.empty_cache() # Nawid - I am not sure if this helps - hopefully it does    
    encoder = ImpalaCNN(observation_channels,all_defaults)
    encoder.load_state_dict(torch.load('/content/MsPacmanNoFrameskip-v4_95.pt'))#,map_location=torch.device('cpu')))
    for param in encoder.parameters():
        param.requires_grad = False 
    #pytorch_total_params = sum(p.numel() for p in encoder.parameters() if p.requires_grad)
    #print(pytorch_total_params)



    encoder.eval()
    
    encoder.to(device)
    #print('does it work here')
    
    # gather the dataset of random sequences
    #rand_dataset = gather_random_trajectories(NUM_RAND_TRAJECTORIES, ENV_NAME)
    rand_dataset = gather_random_trajectories(NUM_RAND_TRAJECTORIES, ENV_NAME, encoder)

    rl_dataset = []
    env = make_vec_envs(ENV_NAME,1,workers,downsample=False) # Nawid- Makes several different vectorised environment
    
    # Initialize the models
    env_model = NNDynamicModel(env.action_space.n + feature_size, feature_size).to(device) # Nawid - Models need to be initialised with the represention size
    #pytorch_total_params = sum(p.numel() for p in env_model.parameters() if p.requires_grad)
    #print('env_model parameters',pytorch_total_params)


    rew_model = NNRewardModel(env.action_space.n + feature_size, 1).to(device)
    #reward_total_params = sum(p.numel() for p in rew_model.parameters() if p.requires_grad)
    #print('reward_model parameters',reward_total_params)
    
    
    game_reward = 0
    num_examples_added = len(rand_dataset)

    for n_iter in range(AGGR_ITER):

        # supervised training of the dataset (random and rl if it exists)
        train_env_loss, train_rew_loss, valid_env_loss, valid_rew_loss, norm = train_dyna_model(rand_dataset, rl_dataset, env_model, rew_model, BATCH_SIZE, TRAIN_ITER_MODEL, num_examples_added, ENV_LEARNING_RATE, REW_LEARNING_RATE, device)
        print('{} >> Eloss:{:.4f} EV loss:{:.4f} -- Rloss:{:.4f} RV loss:{:.4f}'.format(n_iter, train_env_loss, valid_env_loss, train_rew_loss, valid_rew_loss))

        obs = env.reset()

        num_examples_added = 0
        game_reward = 0
        game_pred_rews = []
        rews = []

        while num_examples_added < STEPS_PER_AGGR:
            while True:

                tt = time.time()
                # Execute the control to roll the sequences and pick the first action of the sequence with the higher reward
                action, pred_rew = multi_model_based_control(env_model, rew_model, obs, NUM_ACTIONS_SEQUENCES, HORIZON_LENGTH, env.action_space.sample, norm, device)
                game_pred_rews.append(pred_rew)

                # one step in the environment with the action returned by the controller
                new_obs, reward, done, _ = env.step(action)

                input_scaler, env_output_scaler, rew_output_scaler = norm

                ## Compute the reward and print some stats
                action = np.expand_dims(action,axis=0) # Nawid-  Need to use this to make the dimensions the same during the concatenation   
                models_input = input_scaler.transform([np.concatenate([obs, action])])
                rew_model.eval()
                p_rew = rew_model(torch.tensor(models_input).to(device))
                rew_model.train(True)
                unnorm_rew = rew_output_scaler.inverse_transform([float(p_rew.cpu().data[0])]).squeeze()
                print('  >> ',len(game_pred_rews), 'gt:',np.round(reward,3), 'pred:',np.round(unnorm_rew, 3),
                      'sum:', np.round(pred_rew,3), '|', game_reward, np.round(time.time()-tt, 4), HORIZON_LENGTH)

                # add the last step to the RL dataset
                rl_dataset.append([obs, new_obs, reward, done, action])


                num_examples_added += 1
                obs = new_obs
                game_reward += reward

                # if the environment is done, reset it and print some stats
                if done:
                    obs = env.reset()
                    print('  >> R: {:.2f}, Mean sum:{:.2f}, {}'.format(game_reward, np.mean(game_pred_rews), num_examples_added))

                    rews.append(game_reward)
                    game_reward = 0
                    game_pred_rews = []
                    break

        print('  >> Mean: {:.2f}', np.mean(rews))

main()

0
1
Mean R: 11.0 Max R: 1.0 194.0


  0%|          | 0/55 [00:00<?, ?it/s]

len(D): 388 len(Dtrain) 311
does it work at this point
X_train torch.Size([311, 257])
y_env_train torch.Size([311, 256])
y_rew_train torch.Size([311, 1])
does it work after setting up the scaling
Does it work before making batches


RuntimeError: ignored